# LDA

In [1]:
import pandas as pd
import os
import csv

# Identify the working directory and data files
working_directory = './28-LDA'

# Create the working directory if needed
try:
    os.makedirs(working_directory, exist_ok=True)
except OSError as error:
    print(f"Error creating {working_directory}: {error}")

In [2]:
%pip install --quiet --upgrade nltk gensim spacy pyldavis

Note: you may need to restart the kernel to use updated packages.


In [3]:
!python -m nltk.downloader --quiet 'all'
!python -m spacy download en_core_web_sm > /dev/null

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))


In [4]:
# Read the preprocessed data into a dataframe
import pickle

with open('./21-data_preprocessing/dataframe.pickle', 'rb') as f:
    df = pickle.load(f)

In [5]:
df.shape

(5736, 20)

In [6]:
df.head(1)

,ROW_ID,FOI_TEXT,DEVICE_PROBLEM_CODE,DEVICE_PROBLEM_TEXT,GENERIC_NAME,DEVICE_REPORT_PRODUCT_CODE,UDI-DI,UDI-PUBLIC,DATE_OF_EVENT,REPORTER_OCCUPATION_CODE,REPORT_DATE,EVENT_LOCATION,SOURCE_TYPE,TOKENIZED_TEXT,NOPUNCT_TEXT,NOSTOPWORDS_TEXT,NODIGITS_TEXT,POS_TEXT,LEMMATIZED_TEXT,STEMMED_TEXT
0,1969025,IT WAS REPORTED THAT THE TRANSMITTER LOST CONN...,3283,Wireless Communication Problem,CONTINUOUS GLUCOSE MONITOR,QBJ,,,07/30/2020,000,,I,CONSUMER,"[it, was, reported, that, the, transmitter, lo...","[it, was, reported, that, the, transmitter, lo...","[reported, transmitter, lost, connection, pump...","[reported, transmitter, lost, connection, pump...","[(reported, VBN), (transmitter, NN), (lost, VB...","[report, transmitter, lose, connection, pump, ...","[report, transmitt, lost, connect, pump, great..."


In [7]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = df['FOI_TEXT'].tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['it', 'was', 'reported', 'that', 'the', 'transmitter', 'lost', 'connection', 'with', 'the', 'pump', 'for', 'greater', 'than', 'hour', 'the', 'transmitter', 'ultimately', 'regained', 'connection', 'with', 'the', 'pump', 'no', 'additional', 'patient', 'or', 'event', 'information', 'was']


In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [9]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['report', 'transmitter', 'lose', 'connection', 'pump', 'great', 'hour', 'transmitter', 'ultimately', 'regain', 'connection', 'pump', 'additional', 'patient', 'event', 'information', 'available']


In [11]:
data_words_bigrams[0]

['reported',
 'transmitter',
 'lost',
 'connection',
 'pump',
 'greater',
 'hour',
 'transmitter',
 'ultimately',
 'regained',
 'connection',
 'pump',
 'additional',
 'patient',
 'event',
 'information',
 'available']

In [12]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 2), (13, 1)]


In [13]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [14]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.089*"perform" + 0.060*"pass" + 0.059*"transmitter" + 0.052*"fail" + '
  '0.052*"report" + 0.037*"evaluate" + 0.034*"error" + 0.033*"test" + '
  '0.032*"inspection" + 0.032*"visual"'),
 (1,
  '0.091*"transmitter" + 0.071*"pump" + 0.068*"connection" + 0.065*"patient" + '
  '0.051*"report" + 0.051*"information" + 0.051*"additional" + 0.051*"event" + '
  '0.049*"available" + 0.045*"lose"'),
 (2,
  '0.056*"report" + 0.046*"device" + 0.040*"system" + 0.036*"support" + '
  '0.035*"sensor" + 0.032*"loss" + 0.026*"share" + 0.025*"mobile" + '
  '0.024*"occur" + 0.024*"indicate"'),
 (3,
  '0.096*"loss" + 0.083*"signal" + 0.076*"report" + 0.052*"hour" + '
  '0.039*"probable" + 0.039*"cause" + 0.038*"allegation" + '
  '0.038*"intervention" + 0.038*"occur" + 0.038*"medical"'),
 (4,
  '0.096*"glucose" + 0.077*"bg" + 0.058*"reading" + 0.056*"read" + '
  '0.047*"blood" + 0.044*"meter" + 0.041*"monitor" + 0.041*"continuous" + '
  '0.041*"inaccuracy" + 0.039*"sensor"'),
 (5,
  '0.083*"report" +

In [15]:
from gensim.models import CoherenceModel

# Compute Coherence Score
# TODO: Pickle these as the baseline models
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.6119352495876512


In [16]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    # TODO: Pickle each lda_model and coherence_model_lda
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [17]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 1
max_topics = 25
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {
    'Validation_Set': [],
    'Topics': [],
    'Alpha': [],
    'Beta': [],
    'Coherence': []
}

# Can take a long time to run
if True:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv(f"{working_directory}/lda_tuning_results.csv", index=False)
    pbar.close()

 39%|███▉      | 568/1440 [48:37<58:34,  4.03s/it]  /home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))
100%|██████████| 1440/1440 [2:05:15<00:00,  5.22s/it] 


In [18]:
model_results_df = pd.DataFrame(model_results)

In [26]:
import pickle
import datetime

TAG = f"{datetime.datetime.now():%Y-%m-%d-%s}"

with open(f"{working_directory}/model_results_df-{max_topics}-topics-{TAG}.pickle", 'wb') as f:
    pickle.dump(model_results_df, f)

In [20]:
# TODO: Graph the coherence scores based on some common criteria
model_results_df

,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,1,0.01,0.01,0.573277
1,75% Corpus,1,0.01,0.31,0.573277
2,75% Corpus,1,0.01,0.61,0.573277
3,75% Corpus,1,0.01,0.91,0.573277
4,75% Corpus,1,0.01,symmetric,0.573277
...,...,...,...,...,...
1435,100% Corpus,24,asymmetric,0.01,0.574216
1436,100% Corpus,24,asymmetric,0.31,0.585426
1437,100% Corpus,24,asymmetric,0.61,0.539901
1438,100% Corpus,24,asymmetric,0.91,0.524498


In [21]:
num_topics = 8
alpha      = 0.01
beta       = 0.9

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=alpha,
                                           eta=beta)

In [28]:
import pickle 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join(f"{working_directory}/ldavis_tuned_{num_topics}-topics-{TAG}.pickle")

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if True:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, f"{working_directory}/ldavis_tuned_{num_topics}-topics-{TAG}"+ str(num_topics) +'.html')

LDAvis_prepared

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.248330 -0.002607       1        1  28.030108
6     -0.221258  0.029571       2        1  21.360646
7     -0.184868 -0.114537       3        1  19.671028
0      0.082465 -0.154392       4        1   8.032058
4      0.223930 -0.155618       5        1   7.360167
5      0.180462  0.263301       6        1   6.203053
2     -0.021266  0.193101       7        1   5.298578
1      0.188866 -0.058818       8        1   4.044362, topic_info=            Term         Freq        Total Category  logprob  loglift
12   transmitter  3670.000000  3670.000000  Default  30.0000  30.0000
19    evaluation  2241.000000  2241.000000  Default  29.0000  29.0000
47        device  1084.000000  1084.000000  Default  28.0000  28.0000
26       product  2315.000000  2315.000000  Default  27.0000  27.0000
17         datum  2888.000000  2888.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
185      support    35.925183   254.867019   Topic8  -4.9465   1.2485
143     multiple    28.336623   161.613952   Topic8  -5.1838   1.4668
204   reportedly    32.571700   346.817824   Topic8  -5.0445   0.8425
24         occur    32.673782  3886.853162   Topic8  -5.0414  -1.5709
22          loss    27.179312  3023.782590   Topic8  -5.2255  -1.5040

[405 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
341       1  0.018207  accurate
341       2  0.018207  accurate
341       3  0.764697  accurate
341       4  0.018207  accurate
341       5  0.145657  accurate
...     ...       ...       ...
173       4  0.003995      zone
173       5  0.003995      zone
173       6  0.003995      zone
173       7  0.003995      zone
173       8  0.003995      zone

[2032 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 8, 1, 5, 6, 3, 2])

In [29]:
import os
import fnmatch
import subprocess

bucket = "praxis-2023-html-output"
website = f"http://{bucket}.s3-website-us-west-2.amazonaws.com"

# Use the fnmatch module to find all files in the current directory that end in ".html"
file_list = []
for root, dirnames, filenames in os.walk("."):
    for filename in fnmatch.filter(filenames, '*.html'):
        file_list.append(os.path.join(root, filename))

# Sort the file list alphabetically
file_list.sort()

# Create the HTML file and write the header
with open(os.path.join(".", 'index.html'), 'w') as f:
    f.write('''<html>
        <head>
            <title>Praxis 2023 HTML Output</title>
            <style>
                table {
                    border-collapse: collapse;
                    width: 100%;
                }
                th, td {
                    text-align: left;
                    padding: 8px;
                }
                th {
                    background-color: #007bff;
                    color: #fff;
                    font-weight: bold;
                }
                tr:nth-child(even) {
                    background-color: #f2f2f2;
                }
                tr:hover {
                    background-color: #ddd;
                }
            </style>
        </head>
        <body>
            <table>
                <tr><th>Name</th><th>Size</th></tr>\n
    ''')

    # Loop through each file and add a row to the table
    for file_name in file_list:
        if file_name in ['./index.html']:
            continue
            
        file_size = os.path.getsize(file_name)
        f.write(f'<tr><td><a href="{website}/{file_name}" target="_blank" rel="noopener noreferrer">{file_name}</a></td><td>{int(file_size / 1048576)} MB</td></tr>\n')

    # Write the footer and close the file
    f.write('</table></body></html>')

command = ["aws", "s3", "sync", ".", f"s3://{bucket}", "--exclude", "*", "--include", "*.html", "--no-progress"]

# Run the command and wait for it to complete
output = subprocess.run(command, capture_output=True, text=True)

# Print the output
print(output.stdout)
print('fin')

upload: 28-LDA/ldavis_tuned_8-topics-2023-03-21-16794098878.html to s3://praxis-2023-html-output/28-LDA/ldavis_tuned_8-topics-2023-03-21-16794098878.html
upload: ./index.html to s3://praxis-2023-html-output/index.html

fin
